In [224]:
import pandas as pd
import numpy as np
import os
import re

In [225]:
files = [f"3_pool_GA_CG_GA/reports/{f}" for f in os.listdir("3_pool_GA_CG_GA/reports") if "tmax10_processed" in f]
files

['3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_1_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_2_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_3_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_4_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_5_tmax10_processed.csv']

In [226]:
final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'(?:GA|CG)(?:_(?:GA|CG))*(?=_\d+_)'
    match = re.search(pattern, f)

    # 3. Atribuir o nome do experimento
    if match:
        # Se o padrão for encontrado, o resultado é a correspondência inteira
        exp_name = match.group(0)

    df["experiment_name"] = [exp_name] * len(df.log_ids)
    df = df[df.metric.str.contains("time")].pivot_table(index=["experiment_name"],columns=["step"], values="value").reset_index()
    final_df = pd.concat([final_df, df])

print(final_df)
final_df.to_csv("processing_times_report_GA_CG_GA.csv")

step experiment_name  1_initial_solution  2_preprocessing_and_transformations  \
0           GA_CG_GA           33.053907                             0.014089   
0           GA_CG_GA           27.036171                             0.000000   
0           GA_CG_GA           25.266899                             0.000000   
0           GA_CG_GA           24.338550                             0.004788   
0           GA_CG_GA           24.117425                             0.001024   

step  3_first_ga  4_graph_build  5_column_generation  6_second_ga  
0     443.426231       6.208200           138.563323   594.941258  
0     337.089744       5.362156          1851.111720   837.847549  
0     370.976156       5.377090           159.963252   595.348730  
0     333.956769       5.428990           113.679415   611.720383  
0     348.718444       5.290050           110.045494   566.134285  


In [227]:
files = [f"1_CG_GA/reports/{f}" for f in os.listdir("1_CG_GA/reports") if "tmax10_processed" in f]
files

['1_CG_GA/reports/1_CG_GA_10_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_10_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_11_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_11_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_12_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_12_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_13_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_13_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_14_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_14_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_15_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_15_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_16_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_16_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_17_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_17_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_18_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_18_tmax10_

In [228]:
_final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'(?:GA|CG)(?:_(?:GA|CG))*(?=_\d+_)'
    match = re.search(pattern, f)

    # 3. Atribuir o nome do experimento
    if match:
        # Se o padrão for encontrado, o resultado é a correspondência inteira
        exp_name = match.group(0)

    df["experiment_name"] = [exp_name] * len(df.log_ids)
    df = df[df.metric.str.contains("time")].pivot_table(index=["experiment_name"],columns=["step"], values="value").reset_index()
    _final_df = pd.concat([_final_df, df])
_final_df.to_csv("processing_times_report_CG_GA.csv")
print(_final_df)

step experiment_name  1_initial_solution  2_graph_build  3_column_generation  \
0              CG_GA            6.223845       5.572454           106.044915   
0              CG_GA                 NaN            NaN                  NaN   
0              CG_GA           11.455644       7.073497           290.855911   
0              CG_GA                 NaN            NaN                  NaN   
0              CG_GA           11.474146       7.213027           114.380718   
0              CG_GA                 NaN            NaN                  NaN   
0              CG_GA            6.165298       5.626515            94.508991   
0              CG_GA                 NaN            NaN                  NaN   
0              CG_GA            6.840444       5.620456           104.404299   
0              CG_GA                 NaN            NaN                  NaN   
0              CG_GA            6.587352       5.630694            85.851483   
0              CG_GA                 NaN

COLUMN GENERATION METRICS

In [229]:
files = [f"3_pool_GA_CG_GA/reports/{f}" for f in os.listdir("3_pool_GA_CG_GA/reports") if "tmax10_processed" in f]
files

['3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_1_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_2_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_3_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_4_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_5_tmax10_processed.csv']

In [230]:
final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'(?:GA|CG)(?:_(?:GA|CG))*(?=_\d+_)'
    match = re.search(pattern, f)

    # 3. Atribuir o nome do experimento
    if match:
        # Se o padrão for encontrado, o resultado é a correspondência inteira
        exp_name = match.group(0)

    df["experiment_name"] = [exp_name] * len(df.log_ids)
    df = df[df.step.str.contains("column") & ~df.metric.str.contains("time")].pivot_table(index=["experiment_name"],columns=["metric"], values="value").reset_index()
    final_df = pd.concat([final_df, df])

print(final_df)
final_df.to_csv("column_generation_report_GA_CG_GA.csv")

metric experiment_name  number_of_iterations  optimal_value  start_objective  \
0             GA_CG_GA                  13.0  148143.996958    148508.604958   
0             GA_CG_GA                 138.0  105984.373151    147348.574390   
0             GA_CG_GA                  13.0  144811.359865    145597.259196   
0             GA_CG_GA                  13.0  147295.803308    147731.835308   
0             GA_CG_GA                  13.0  147948.481017    148217.054416   

metric  total_columns_generated  total_reduced_cost  
0                         293.0       -4.303187e+06  
0                        3428.0       -8.450975e+06  
0                         292.0       -2.689728e+06  
0                         315.0       -2.718052e+06  
0                         312.0       -3.810087e+06  


In [231]:
files = [f"1_CG_GA/reports/{f}" for f in os.listdir("1_CG_GA/reports") if "tmax10_processed" in f]
files

['1_CG_GA/reports/1_CG_GA_10_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_10_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_11_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_11_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_12_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_12_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_13_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_13_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_14_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_14_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_15_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_15_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_16_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_16_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_17_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_17_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_18_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_18_tmax10_

In [232]:
_final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'(?:GA|CG)(?:_(?:GA|CG))*(?=_\d+_)'
    match = re.search(pattern, f)

    # 3. Atribuir o nome do experimento
    if match:
        # Se o padrão for encontrado, o resultado é a correspondência inteira
        exp_name = match.group(0)

    df["experiment_name"] = [exp_name] * len(df.log_ids)
    df = df[df.step.str.contains("column") & ~df.metric.str.contains("time")].pivot_table(index=["experiment_name"],columns=["metric"], values="value").reset_index()
    _final_df = pd.concat([_final_df, df])

print(_final_df)
_final_df.to_csv("column_generation_report_CG_GA.csv")

metric experiment_name  number_of_iterations  optimal_value  start_objective  \
0                CG_GA                  16.0  146619.464351    147434.504351   
0                CG_GA                  16.0  146619.464351    147434.504351   
0                CG_GA                  30.0  149951.760080    151670.100080   
0                CG_GA                  30.0  149951.760080    151670.100080   
0                CG_GA                  13.0  144565.357925    145029.613925   
0                CG_GA                  13.0  144565.357925    145029.613925   
0                CG_GA                  14.0  148761.489785    149855.889785   
0                CG_GA                  14.0  148761.489785    149855.889785   
0                CG_GA                  15.0  149055.889255    150351.746654   
0                CG_GA                  15.0  149055.889255    150351.746654   
0                CG_GA                  13.0  149577.778067    150222.898067   
0                CG_GA                  

GENETIC ALGORITHM

In [233]:
files = [f"3_pool_GA_CG_GA/reports/{f}" for f in os.listdir("3_pool_GA_CG_GA/reports") if "tmax10_processed" in f]
files

['3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_1_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_2_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_3_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_4_tmax10_processed.csv',
 '3_pool_GA_CG_GA/reports/3_pool_GA_CG_GA_5_tmax10_processed.csv']

In [234]:
final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'(?:GA|CG)(?:_(?:GA|CG))*(?=_\d+_)'
    match = re.search(pattern, f)

    # 3. Atribuir o nome do experimento
    if match:
        # Se o padrão for encontrado, o resultado é a correspondência inteira
        exp_name = match.group(0)

    df["experiment_name"] = [exp_name] * len(df.log_ids)
    df = df[df.step.str.contains("second") & ~df.metric.str.contains("time")].pivot_table(index=["experiment_name"],columns=["metric"], values="value").reset_index()
    final_df = pd.concat([final_df, df])

print(final_df)
final_df.to_csv("ga_report_GA_CG_GA.csv")

metric experiment_name      best_cost  best_fitness  best_r  best_s  best_u  \
0             GA_CG_GA  206389.234571      0.088593    35.0   137.0    13.0   
0             GA_CG_GA  184233.996191      0.066033   101.0    76.0    15.0   
0             GA_CG_GA  191501.347970      0.105273    48.0   112.0     8.0   
0             GA_CG_GA  204872.219776      0.077856    52.0   129.0    13.0   
0             GA_CG_GA  194340.313153      0.098107    43.0   117.0    12.0   

metric  best_v  
0        180.0  
0        140.0  
0        160.0  
0        176.0  
0        164.0  


In [235]:
files = [f"1_CG_GA/reports/{f}" for f in os.listdir("1_CG_GA/reports") if "tmax10_processed" in f]
files

['1_CG_GA/reports/1_CG_GA_10_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_10_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_11_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_11_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_12_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_12_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_13_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_13_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_14_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_14_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_15_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_15_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_16_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_16_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_17_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_17_tmax10_processed_processed.csv',
 '1_CG_GA/reports/1_CG_GA_18_tmax10_processed.csv',
 '1_CG_GA/reports/1_CG_GA_18_tmax10_

In [236]:
_final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'(?:GA|CG)(?:_(?:GA|CG))*(?=_\d+_)'
    match = re.search(pattern, f)

    # 3. Atribuir o nome do experimento
    if match:
        # Se o padrão for encontrado, o resultado é a correspondência inteira
        exp_name = match.group(0)

    df["experiment_name"] = [exp_name] * len(df.log_ids)
    df = df[df.step.str.contains("ga") & ~df.metric.str.contains("time")].pivot_table(index=["experiment_name"],columns=["metric"], values="value").reset_index()
    _final_df = pd.concat([_final_df, df])

print(_final_df)
_final_df.to_csv("ga_report_CG_GA.csv")

metric experiment_name      best_cost  best_fitness  best_r  best_s  best_u  \
0                CG_GA  110135.864088      9.960159    10.0    32.0    61.0   
0                CG_GA  110135.864088      9.960159    10.0    32.0    61.0   
0                CG_GA  113650.545032      5.537099    51.0    39.0   143.0   
0                CG_GA  113650.545032      5.537099    51.0    39.0   143.0   
0                CG_GA  115580.177287     10.869565     3.0    38.0    44.0   
0                CG_GA  115580.177287     10.869565     3.0    38.0    44.0   
0                CG_GA  112246.900006     10.989011     0.0    40.0    46.0   
0                CG_GA  112246.900006     10.989011     0.0    40.0    46.0   
0                CG_GA  107748.098958     11.235955     0.0    32.0    54.0   
0                CG_GA  107748.098958     11.235955     0.0    32.0    54.0   
0                CG_GA  111621.132563      8.896797    22.0    32.0    69.0   
0                CG_GA  111621.132563      8.896797 

In [171]:
import pandas as pd
import os
import glob

def rename_ga_blocks(df):
    """
    Identifica blocos de genetic_algorithm e renomeia para first_ga e second_ga
    """
    df = df.copy()
    
    # Encontrar todas as linhas de genetic_algorithm
    ga_mask = df['step'] == 'genetic_algorithm'
    
    if not ga_mask.any():
        return df
    
    # Extrair parte numérica dos log_ids para identificar grupos
    ga_rows = df[ga_mask].copy()

    # --- LINHA CORRIGIDA ---
    # Extrai a parte numérica da string e converte para float para evitar OverflowError
    ga_rows['log_id_base'] = ga_rows['log_ids'].astype(str).str.extract(r'(\d+)')[0].astype(float)
    
    # Identificar quebras significativas nos log_ids (indicando nova execução do GA)
    ga_rows['id_diff'] = ga_rows['log_id_base'].diff()
    
    # Uma diferença grande nos IDs indica uma nova execução
    # Vamos usar um threshold baseado na observação dos dados
    threshold = 10000000  # 10 milhões - ajustar se necessário
    
    # Marcar onde começam novos blocos de GA
    new_block_mask = (ga_rows['id_diff'] > threshold) | (ga_rows['id_diff'].isna())
    ga_rows['block_number'] = new_block_mask.cumsum() - 1
    
    # Renomear as steps baseado no número do bloco
    for idx, row in ga_rows.iterrows():
        if row['block_number'] == 0:
            df.loc[idx, 'step'] = 'first_ga'
        elif row['block_number'] == 1:
            df.loc[idx, 'step'] = 'second_ga'
        else:
            df.loc[idx, 'step'] = f'ga_{int(row["block_number"]) + 1}'
    
    return df

def add_step_numbering(df):
    """
    Adiciona numeração sequencial às steps mantendo o mesmo número para steps do mesmo tipo
    """
    df = df.copy()
    
    # Identificar a ordem das steps únicas mantendo a sequência original
    step_order = []
    seen_steps = set()
    
    for step in df['step']:
        if step not in seen_steps:
            step_order.append(step)
            seen_steps.add(step)
    
    # Criar mapeamento de step para número
    step_to_number = {step: i + 1 for i, step in enumerate(step_order)}
    
    # Aplicar numeração
    df['numbered_step'] = df['step'].map(step_to_number).astype(str) + '_' + df['step']
    df['step'] = df['numbered_step']
    df.drop('numbered_step', axis=1, inplace=True)
    
    return df

def process_csv_file(file_path):
    """
    Processa um arquivo CSV aplicando as transformações solicitadas:
    1. Substitui experiment_name baseado no nome do arquivo
    2. Renomeia blocos de GA quando há múltiplas execuções
    3. Adiciona numeração sequencial às steps
    """
    
    # Ler o CSV
    df = pd.read_csv(file_path)
    
    # 1. Transformar o nome do experimento baseado no arquivo
    filename = file_path.split("/")[-1]
    filename = filename.split(".")[-2]  # Remove extensão
    exp_name = "_".join(filename.split("_")[0:-1])
    
    df["experiment_name"] = exp_name
    
    # 2. Identificar e renomear blocos de GA
    df = rename_ga_blocks(df)
    
    # 3. Adicionar numeração sequencial às steps
    df = add_step_numbering(df)
    
    return df

def process_single_csv(file_path, save_output=True):
    """
    Função principal simplificada para processar um único arquivo CSV
    
    Args:
        file_path (str): Caminho para o arquivo CSV
        save_output (bool): Se deve salvar o arquivo processado
    
    Returns:
        DataFrame: DataFrame processado
    """
    print(f"Processando arquivo: {file_path}")
    
    # Processar arquivo
    df = process_csv_file(file_path)
    
    print(f"Dados processados: {len(df)} linhas")
    
    # Mostrar informações do processamento
    filename = file_path.split("/")[-1].split(".")[-2]
    exp_name = "_".join(filename.split("_")[0:-1])
    print(f"Novo experiment_name: {exp_name}")
    
    # Contar blocos de GA
    first_ga_count = len(df[df['step'].str.contains('first_ga', na=False)])
    second_ga_count = len(df[df['step'].str.contains('second_ga', na=False)])
    
    if first_ga_count > 0:
        print(f"Blocos GA identificados - first_ga: {first_ga_count}, second_ga: {second_ga_count}")
    
    # Mostrar steps finais
    unique_steps = df['step'].unique()
    print(f"Steps finais: {list(unique_steps)}")
    
    # Salvar se solicitado
    if save_output:
        output_path = file_path.replace('.csv', '_processed.csv')
        df.to_csv(output_path, index=False)
        print(f"Arquivo salvo: {output_path}")
    
    return df

def process_multiple_files(file_paths):
    """
    Processa múltiplos arquivos CSV
    """
    processed_dfs = []
    successful = 0
    failed = 0
    
    for file_path in file_paths:
        df_processed = process_single_csv(file_path)
        if df_processed is not None:
            processed_dfs.append(df_processed)
            successful += 1
            print("✓ Processado com sucesso\n")
        else:
            failed += 1
            print("✗ Falhou no processamento\n")
    
    print(f"RESUMO: {successful} arquivos processados com sucesso, {failed} falharam")
    return processed_dfs

def process_multiple_files(file_paths):
    """
    Processa múltiplos arquivos CSV
    """
    processed_dfs = []
    successful = 0
    failed = 0
    
    for file_path in file_paths:
        df_processed = process_single_csv(file_path)
        if df_processed is not None:
            processed_dfs.append(df_processed)
            successful += 1
            print("✓ Processado com sucesso\n")
        else:
            failed += 1
            print("✗ Falhou no processamento\n")
    
    print(f"RESUMO: {successful} arquivos processados com sucesso, {failed} falharam")
    return processed_dfs

# ===================== EXEMPLOS DE USO =====================

if __name__ == "__main__":
    
    # EXEMPLO 1: Processar um arquivo único
    # df = process_single_csv("caminho/para/seu_arquivo.csv")
    
    # EXEMPLO 2: Processar múltiplos arquivos de uma pasta
    arquivos = glob.glob("1_CG_GA/reports/*.csv")
    dfs_processados = process_multiple_files(arquivos)
    
    # EXEMPLO 3: Processar sem salvar automaticamente
    # df = process_single_csv("arquivo.csv", save_output=False)
    
    # EXEMPLO 4: Processar todos os CSV do diretório atual
    # arquivos_locais = glob.glob("*.csv")
    # for arquivo in arquivos_locais:
    #     process_single_csv(arquivo)
    
    print("Para usar este código:")
    print("1. Arquivo único: process_single_csv('caminho/arquivo.csv')")
    print("2. Múltiplos arquivos: process_multiple_files(['arquivo1.csv', 'arquivo2.csv'])")
    print("3. Todos CSV da pasta: process_multiple_files(glob.glob('pasta/*.csv'))")

Processando arquivo: 1_CG_GA/reports\1_CG_GA_10_tmax10.csv
Dados processados: 18 linhas
Novo experiment_name: reports\1_CG_GA_10
Blocos GA identificados - first_ga: 7, second_ga: 0
Steps finais: ['1_initial_solution', '2_graph_build', '3_column_generation', '4_first_ga']
Arquivo salvo: 1_CG_GA/reports\1_CG_GA_10_tmax10_processed.csv
✓ Processado com sucesso

Processando arquivo: 1_CG_GA/reports\1_CG_GA_10_tmax10_processed.csv
Dados processados: 18 linhas
Novo experiment_name: reports\1_CG_GA_10_tmax10
Blocos GA identificados - first_ga: 7, second_ga: 0
Steps finais: ['1_1_initial_solution', '2_2_graph_build', '3_3_column_generation', '4_4_first_ga']
Arquivo salvo: 1_CG_GA/reports\1_CG_GA_10_tmax10_processed_processed.csv
✓ Processado com sucesso

Processando arquivo: 1_CG_GA/reports\1_CG_GA_11_tmax10.csv
Dados processados: 18 linhas
Novo experiment_name: reports\1_CG_GA_11
Blocos GA identificados - first_ga: 7, second_ga: 0
Steps finais: ['1_initial_solution', '2_graph_build', '3_colum